In [1]:
%load_ext autoreload

In [119]:
import polars as pl
from skolegpt_instruct_dataset.config import config 
from skolegpt_instruct_dataset.filtering import remove_duplicate_questions_and_responses
df = pl.read_parquet(config.data_dir / config.stratified_dataset_file_name)

In [122]:
def translate_system_prompts(df: pl.DataFrame) -> pl.DataFrame:
    unique_sys_prompts = None
    return

id,system_prompt,question,response,source
str,str,str,str,str
"""cot.9768""","""You are an AI …","""Stream of cons…","""Question: Can …","""cot"""
"""cot.119533""","""You are an AI …","""Premise: ""A ma…","""The premise de…","""cot"""
"""cot.54006""","""You are an AI …","""Leo: Premise: …","""Based on the i…","""cot"""
"""cot.120822""","""You are an AI …","""Premise: ""A ma…","""The hypothesis…","""cot"""
"""cot.139399""","""You are an AI …","""Premise: ""The …","""Yes, the hypot…","""cot"""
"""cot.24910""","""You are an AI …","""Stream of cons…","""Question: What…","""cot"""
"""cot.16473""","""You are an AI …","""Given the sent…","""Based on the s…","""cot"""
"""cot.14152""","""You are an AI …","""Premise: ""Thre…","""In this scenar…","""cot"""
"""cot.49448""","""You are an AI …","""On Friday, Han…","""Sure! We can b…","""cot"""
